In [ ]:
!pip install -q python-crfsuite
!pip install -q  pythainlp

     |████████████████████████████████| 743 kB 3.7 MB/s 
     |████████████████████████████████| 11.5 MB 4.3 MB/s 


In [ ]:
! pip install -q datasets transformers[sentencepiece] simpletransformers

     |████████████████████████████████| 311 kB 5.6 MB/s 
     |████████████████████████████████| 3.5 MB 35.6 MB/s 
     |████████████████████████████████| 248 kB 48.4 MB/s 
     |████████████████████████████████| 133 kB 35.4 MB/s 
     |████████████████████████████████| 1.1 MB 47.2 MB/s 
     |████████████████████████████████| 243 kB 47.0 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
     |████████████████████████████████| 6.8 MB 31.9 MB/s 
     |████████████████████████████████| 1.7 MB 27.3 MB/s 
     |████████████████████████████████| 1.2 MB 27.3 MB/s 
     |████████████████████████████████| 43 kB 1.4 MB/s 
     |████████████████████████████████| 9.7 MB 22.6 MB/s 
     |████████████████████████████████| 596 kB 10.4 MB/s 
     |████████████████████████████████| 895 kB 35.8 MB/s 
     |████████████████████████████████| 181 kB 38.9 MB/s 
     |████████████████████████████████| 144 kB 26.4 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
     |███████████████

In [ ]:
from datasets import load_dataset
lst20 = load_dataset("lst20", data_dir="/content/drive/MyDrive/super-AI-level2/Hackathon-NLP/Data/LST20-Data-full/LST20_Corpus")

In [ ]:
_POS_TAGS = ["NN", "VV", "PU", "CC", "PS", "AX", "AV", "FX", "NU", "AJ", "CL", "PR", "NG", "PA", "XX", "IJ"]

In [ ]:
import pandas as pd
train_df = pd.DataFrame(lst20['train'])
validation_df = pd.DataFrame(lst20['validation'])
test_df = pd.DataFrame(lst20['test'])

In [ ]:
def convert_to_simple_transformer_format(df, field_name, tags):
  sentence_id = []
  words = []
  labels = []

  #Limit at 1000 rows for speed.
  for (idx, r) in df[:].iterrows():
    # print(idx)
    for (i, t) in enumerate(r['tokens']):
      # print(i,t)
      sentence_id.append(idx)
      words.append(t)
      labels.append(tags[r[field_name][i]])

  return pd.DataFrame(
      {"sentence_id": sentence_id, "words": words, "labels": labels}
  )

In [ ]:
train = convert_to_simple_transformer_format(train_df, "pos_tags", _POS_TAGS)

In [ ]:
validation = convert_to_simple_transformer_format(validation_df, "pos_tags", _POS_TAGS)
test = convert_to_simple_transformer_format(test_df, "pos_tags", _POS_TAGS)

In [ ]:
#แทนที่ _ ด้วย <space>
train["words"].replace({"_": "<space>"}, inplace=True)
test["words"].replace({"_": "<space>"}, inplace=True)
validation["words"].replace({"_": "<space>"}, inplace=True)
    

In [ ]:
len(train),len(test),len(validation)

(2714726, 207278, 240860)

In [ ]:
def sentenceStop(train_df):
  numCounter = []
  counter = 0
  for i in range(len(train_df)):
    counter = counter + len(train_df['clause_tags'][i])
    numCounter.append(counter)
  return numCounter
   

In [ ]:
def sentenceLenght(train_df):
  numStop = []
  counterStop = 0
  for i in range(len(train_df)):
    counterStop =  len(train_df['clause_tags'][i])
    numStop.append(counterStop)
  return numStop

In [ ]:
#จำนวนข้อความทั้งหมด ตัดแบ่งตามจุดจบประโยค
a = sentenceLenght(train_df)
b = sentenceStop(train_df)

In [ ]:
a[0:5],b[0:5]

([12, 9, 26, 20, 8], [12, 21, 47, 67, 75])

In [ ]:
test_df[0:12]

['อย่างไรก็ตาม',
 'เครื่อง',
 'บิน',
 'แอร์บัส',
 '',
 'เอ',
 '380',
 'ได้',
 'ถึง',
 'ที่',
 'หมาย',
 'ที่']

In [ ]:
def dataToCRFformat(train,numCounter,numStop):
  temp = []
  datas = []
  countLoop = 0
  for t in range(len(numCounter)):
    if t == 0:
      for i in range(0,numCounter[t]):
        #  print(i)
        a = train['words'][i]
        b = train['labels'][i]
        c = (a,b)
        temp.append(c)

    else:
      for k in range(numStop[t-1], numStop[t]):
        # print(i)
        a = train['words'][k]
        b = train['labels'][k]
        c = (a,b)
        temp.append(c)
    datas.append(temp)
    temp = []
  return datas


In [ ]:
#call function formatting data
datas_train = dataToCRFformat(train,sentenceLenght(train_df),sentenceStop(train_df))

In [ ]:
datas_validate = dataToCRFformat(validation,sentenceLenght(validation_df),sentenceStop(validation_df))

In [ ]:
datas_test = dataToCRFformat(test,sentenceLenght(test_df),sentenceStop(test_df))

In [ ]:
datas = datas_train + datas_validate + datas_test

In [ ]:
len(datas)

74180

In [ ]:
%%time
from nltk.tag import CRFTagger
ct = CRFTagger()
ct.train(datas,'model.crf.tagger') 

CPU times: user 30min 51s, sys: 9.33 s, total: 31min
Wall time: 30min 51s


In [ ]:
# %%time
# from nltk.tag import CRFTagger
# from pythainlp.tokenize import word_tokenize
# ct = CRFTagger()
# ct.set_model_file('/content/model.crf.tagger')
# text=""
# # while text!="exit":
# #  text=input("Text : ")
# post=word_tokenize('อย่างไรก็ตาม เครื่อง บิน แอร์บัส เอ 380')

# print(ct.tag_sents([post]))

[[('อย่างไรก็ตาม', 'CC'), (' ', 'VV'), ('เครื่อง', 'NN'), (' ', 'NN'), ('บิน', 'VV'), (' ', 'PS'), ('แอร์บัส', 'NN'), (' ', 'PS'), ('เอ', 'NN'), (' ', 'AJ'), ('380', 'NU')]]
CPU times: user 3.04 ms, sys: 2.02 ms, total: 5.06 ms
Wall time: 7.43 ms


In [ ]:
f = open("/content/pos_test.txt", "r")
test_data = f.read()

In [ ]:
test_data = test_data.replace("\n", " ")
test_pos = test_data.split(" ")

In [ ]:
# a = test_df[0:50000]

In [ ]:
%%time
result = ct.tag_sents([test_pos])

CPU times: user 854 ms, sys: 211 ms, total: 1.07 s
Wall time: 1.07 s


In [ ]:
result[0]

[('อย่างไรก็ตาม', 'CC'),
 ('เครื่อง', 'NN'),
 ('บิน', 'VV'),
 ('แอร์บัส', 'NN'),
 ('', 'PS'),
 ('เอ', 'NN'),
 ('380', 'NU'),
 ('ได้', 'AX'),
 ('ถึง', 'VV'),
 ('ที่', 'PS'),
 ('หมาย', 'NN'),
 ('ที่', 'PS'),
 ('ท่าอากาศยาน', 'NN'),
 ('เชียงใหม่', 'NN'),
 ('ใน', 'PS'),
 ('เวลา', 'NN'),
 ('', 'AJ'),
 ('14.50', 'NU'),
 ('', 'NU'),
 ('น.', 'CL'),
 ('โดย', 'CC'),
 ('ใช้', 'VV'),
 ('เวลา', 'NN'),
 ('บิน', 'VV'),
 ('ประมาณ', 'AJ'),
 ('1', 'NU'),
 ('ชั่วโมง', 'CL'),
 ('', 'PS'),
 ('มูลนิธิสืบฯ', 'NN'),
 ('พอใจ', 'VV'),
 ('ฟื้น', 'VV'),
 ('ป่าห้วยขาแข้ง', 'NN'),
 ('มี', 'VV'),
 ('เสือโคร่ง', 'VV'),
 ('มาก', 'VV'),
 ('อันดับ', 'NN'),
 ('2', 'NU'),
 ('โลก', 'NN'),
 ('', 'AJ'),
 ('1', 'NU'),
 ('', 'PS'),
 ('กันยายน', 'NN'),
 ('', 'AJ'),
 ('2550', 'NU'),
 ('17.31', 'NU'),
 ('น.', 'CL'),
 ('', 'PS'),
 ('ประธาน', 'NN'),
 ('มูลนิธิ', 'NN'),
 ('สืบ', 'VV'),
 ('นาคะเสถียร', 'NN'),
 ('พอใจ', 'VV'),
 ('ป่า', 'NN'),
 ('ห้วยขาแข้ง', 'NN'),
 ('ฟื้นตัว', 'VV'),
 ('พบ', 'VV'),
 ('เสือโคร่ง', 'VV'),
 ('2-3', 'NU'

In [ ]:
summit = []
for re in result[0]:
  if re[0] == '':
    # print(re)
    summit.append('PU')
  else:
    summit.append(re[1])
  

In [ ]:
len(summit)

103915

In [ ]:
final = pd.DataFrame(summit)

In [ ]:
final.reset_index(inplace=True)

In [ ]:
final['index'] = final['index'].apply(lambda x:x+1)

In [ ]:
final

,index,0
0,1,CC
1,2,NN
2,3,VV
3,4,NN
4,5,PU
...,...,...
103910,103911,VV
103911,103912,FX
103912,103913,VV
103913,103914,PU


In [ ]:
final.to_csv('CRFSubmissionTrainnValidTest.csv')